In [1]:
import numpy as np
import pandas as pd
import altair as alt
from altair import datum
import ast
from datetime import datetime


# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable('data_server')
# # Include an image for each plot since Gradescope only supports displaying plots as images
# alt.renderers.enable('mimetype')

DataTransformerRegistry.enable('data_server')

In [2]:
url = "https://github.com/kemiolamudzengi/dsci-320-datasets/blob/main/amazon_conlit_goodreads_nyt.csv?raw=true"
books = pd.read_csv(url, parse_dates=['amazon_year', 'conlit_pubdate', 'nyt_published_date',
                                      'goodreads_publish_date', 'goodreads_first_publish_date'
                                     ] )

## Task 1:  Which year has the highest proportions of books that have more than 3 awards?

### Data wrangling for Task 1

In [3]:
books_awards = books.copy()
# drop column with NaN in goodreads_awards and amazon_year
books_awards = books_awards.dropna(subset=["goodreads_awards", "amazon_year"], how="any")
# drop duplicates title
books_awards = books_awards.drop_duplicates(subset=["title"])
# cast goodreads_awards element as list
books_awards["goodreads_award"] = books_awards.goodreads_awards.apply(lambda x: ast.literal_eval(x))
# create label for more than 3 awards for each book
books_awards["num_awards"] = books_awards["goodreads_award"].apply(lambda x: "More than 3 awards" if len(x) > 3 else "3 awards or less")
# calculate the count of each year-num_awards category combination
books_awards = books_awards.groupby(["amazon_year", "num_awards"])["num_awards"].count().rename("count").reset_index()
# calculate the total number of books in a year
books_awards["total"] = books_awards.groupby("amazon_year")["count"].transform(lambda x: sum(x))
# calculate the respective percentage of each year, round to 2 decimal
books_awards["percentage"] = round(books_awards["count"] / books_awards["total"], 2)
books_awards

,amazon_year,num_awards,count,total,percentage
0,2009-01-01,3 awards or less,28,39,0.72
1,2009-01-01,More than 3 awards,11,39,0.28
2,2010-01-01,3 awards or less,18,28,0.64
3,2010-01-01,More than 3 awards,10,28,0.36
4,2011-01-01,3 awards or less,19,24,0.79
5,2011-01-01,More than 3 awards,5,24,0.21
6,2012-01-01,3 awards or less,20,23,0.87
7,2012-01-01,More than 3 awards,3,23,0.13
8,2013-01-01,3 awards or less,17,25,0.68
9,2013-01-01,More than 3 awards,8,25,0.32


### Chart

In [4]:
award_proportion = alt.Chart(books_awards).mark_bar().encode(
    alt.X("year(amazon_year):O", axis=alt.Axis(title="Year published (Amazon)")),
    alt.Y("percentage", axis=alt.Axis(title="Proportion")),
    alt.Color("num_awards", legend=alt.Legend(title="Number of awards"), 
              scale=alt.Scale(scheme="viridis")),
    alt.Tooltip(["percentage"])
).properties(
    title="Proportion of books with more than 3 awards for each year"
)

award_proportion

alt.Chart(...)

## Task 2 - What are the Most Popular Book Genres Over the Years?

In [5]:
books['conlit_pubyear'] = books['conlit_pubdate'].dt.year

dropdown_options = sorted(
    books['conlit_pubyear'].dropna().unique(), 
    reverse=True
)

year_selector = alt.binding_select(
    options=dropdown_options,
    name = 'Year: '
)
year_dropdown = alt.selection_single(
    fields=['conlit_pubyear'], 
    init={'conlit_pubyear': dropdown_options[0]},
    bind=year_selector
)
hover = alt.selection_single(
    fields=['conlit_genre', 'conlit_pubyear'],
    on='mouseover',  
    nearest=True,    
    empty='none'
)

color_scale = alt.Scale(
    domain=['BIO', 'BS', 'HIST', 'MEM', 'MID', 'MIX', 'NYT', 'PW', 'SF', 'YA'],
    range=['#8dd3c7', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5', '#d9d9d9', '#bc80bd', '#ccebc5']
)


In [6]:
## line chart over time

click = alt.selection_multi(fields = ['conlit_genre'], bind='legend')

base = alt.Chart(books.dropna(subset=['conlit_genre'])).encode(
    alt.X('conlit_pubyear:O', axis=alt.Axis(title="Conlit Publication Year")),
    alt.Y('count()'),
    alt.Color('conlit_genre', scale=color_scale)
).properties(
    width = 600,
    height = 250,
    title = 'Trends in Book Genres Over the Years'
)

genre_line = base.mark_line().add_selection(click).encode(
    opacity=alt.condition(click, alt.value(1), alt.value(0.1))
) + base.mark_text(dx=4, dy=-8, align='right').encode(
    text='conlit_genre',
    opacity=alt.condition(hover, alt.value(1), alt.value(0))
).add_selection(
    hover
) + base.mark_circle().encode(opacity=alt.condition(click, alt.value(1), alt.value(0.1)))

# bar chart
genre_bar = alt.Chart(books.dropna(subset=['conlit_genre'])).transform_filter(
    year_dropdown
).mark_bar().encode(
    alt.X('count()', axis=alt.Axis(tickMinStep=1)),
    alt.Y('conlit_genre', sort='-x',axis=alt.Axis(title="Conlit Genre")),
    alt.Color('conlit_genre', scale=color_scale, legend=alt.Legend(title="Conlit Genre"))
).add_selection(
    year_dropdown
).properties(
    width=600,
    height=200,
    title='Book Genres Published for Each Individual Year'
)

genre_line & genre_bar

alt.VConcatChart(...)

## Task 3: For the Top 10 most popular genres, how many books were published by Top 5 publishers? 

### Data wrangling for Task 3

In [7]:
books_genres = books.copy()
# cast as list element
books_genres["goodreads_genres"] = books_genres["goodreads_genres"].apply(lambda x: ast.literal_eval(x))
# drop rows with NaN value for goodreads_publisher
books_genres_top10 = books_genres.dropna(subset=["goodreads_publisher"])
# explode the dataframe on goodreads_genres column
books_genres_top10 = books_genres_top10.explode("goodreads_genres")
# 
sorted_genres = books_genres_top10["goodreads_genres"].value_counts()[0:10].index.tolist()

books_genres_top10 = books_genres_top10[books_genres_top10.goodreads_genres.apply(lambda x: x in sorted_genres)]
books_genres_top10.head()

,title,amazon_author,amazon_rating,amazon_num_reviews,amazon_price,amazon_year,amazon_genre,conlit_genre,conlit_pubdate,conlit_author_gender,...,goodreads_first_publish_date,goodreads_awards,goodreads_num_ratings,goodreads_likedPercent,goodreads_price,nyt_published_date,nyt_list_name_encoded,nyt_price,nyt_weeks_on_list,conlit_pubyear
0,11/22/63: A Novel,Stephen King,4.6,2052,22,2011-01-01,Fiction,BS,2011-01-01,M,...,NaT,['Locus Award Nominee for Best SF Novel (2012)...,420225.0,96.0,6.21,2012-02-26,hardcover-fiction,35.0,14.0,2011.0
0,11/22/63: A Novel,Stephen King,4.6,2052,22,2011-01-01,Fiction,BS,2011-01-01,M,...,NaT,['Locus Award Nominee for Best SF Novel (2012)...,420225.0,96.0,6.21,2012-02-26,hardcover-fiction,35.0,14.0,2011.0
0,11/22/63: A Novel,Stephen King,4.6,2052,22,2011-01-01,Fiction,BS,2011-01-01,M,...,NaT,['Locus Award Nominee for Best SF Novel (2012)...,420225.0,96.0,6.21,2012-02-26,hardcover-fiction,35.0,14.0,2011.0
1,A Dance with Dragons (A Song of Ice and Fire),George R. R. Martin,4.4,12643,11,2011-01-01,Fiction,BS,2011-01-01,M,...,NaT,"['Hugo Award Nominee for Best Novel (2012)', '...",555900.0,97.0,NaN,NaT,NaN,NaN,NaN,2011.0
1,A Dance with Dragons (A Song of Ice and Fire),George R. R. Martin,4.4,12643,11,2011-01-01,Fiction,BS,2011-01-01,M,...,NaT,"['Hugo Award Nominee for Best Novel (2012)', '...",555900.0,97.0,NaN,NaT,NaN,NaN,NaN,2011.0


In [8]:
books_genres_top5pub = books_genres["goodreads_publisher"].value_counts()[0:5].rename("count").reset_index()
sorted_pubs = books_genres_top5pub["index"].tolist()
books_genres_top5pub

,index,count
0,Random House,13
1,Alfred A. Knopf,12
2,Thomas Nelson,11
3,Scholastic Press,11
4,Northfield Publishing,10


In [9]:
books_genres_publishers = books_genres_top5pub.merge(right=books_genres_top10, left_on="index", 
                                                     right_on="goodreads_publisher", how="left")

books_genres_publishers.head()

,index,count,title,amazon_author,amazon_rating,amazon_num_reviews,amazon_price,amazon_year,amazon_genre,conlit_genre,...,goodreads_first_publish_date,goodreads_awards,goodreads_num_ratings,goodreads_likedPercent,goodreads_price,nyt_published_date,nyt_list_name_encoded,nyt_price,nyt_weeks_on_list,conlit_pubyear
0,Random House,13,Olive Kitteridge,Elizabeth Strout,4.2,4519,12,2009-01-01,Fiction,BS,...,NaT,"['Pulitzer Prize for Fiction (2009)', 'Premio ...",171636.0,90.0,NaN,NaT,NaN,NaN,NaN,2008.0
1,Random House,13,Olive Kitteridge,Elizabeth Strout,4.2,4519,12,2009-01-01,Fiction,BS,...,NaT,"['Pulitzer Prize for Fiction (2009)', 'Premio ...",171636.0,90.0,NaN,NaT,NaN,NaN,NaN,2008.0
2,Random House,13,Olive Kitteridge,Elizabeth Strout,4.2,4519,12,2009-01-01,Fiction,BS,...,NaT,"['Pulitzer Prize for Fiction (2009)', 'Premio ...",171636.0,90.0,NaN,NaT,NaN,NaN,NaN,2008.0
3,Random House,13,When Breath Becomes Air,Paul Kalanithi,4.8,13779,14,2016-01-01,Non Fiction,MEM,...,2016-01-12,['Pulitzer Prize Nominee for Biography or Auto...,415362.0,96.0,NaN,NaT,NaN,NaN,NaN,2016.0
4,Random House,13,When Breath Becomes Air,Paul Kalanithi,4.8,13779,14,2016-01-01,Non Fiction,MEM,...,2016-01-12,['Pulitzer Prize Nominee for Biography or Auto...,415362.0,96.0,NaN,NaT,NaN,NaN,NaN,2016.0


### Chart

In [10]:
heatmap = alt.Chart(books_genres_publishers).mark_rect().encode(
    alt.X("index", axis=alt.Axis(title="Publisher"), 
          sort=sorted_pubs),
    alt.Y("goodreads_genres", axis=alt.Axis(title="Genre (Goodreads)"), 
          sort=sorted_genres),
    alt.Color("count(goodreads_genres)", scale=alt.Scale(scheme="greenblue")),
    alt.Tooltip(["count(goodreads_genres)"])
).properties(
    title="For Top 10 most popular genres, how many books were published by Top 5 publishers",
    width=170,
    height=340
)

heatmap

alt.Chart(...)

## Task 4: Do fiction books stay on the NYT best-selling list longer than non-fiction books and are they more popular on Amazon and Goodreads?

In [11]:
books["total_star_amazon"] = books["amazon_rating"] * books["amazon_num_reviews"]
books["total_star_goodreads"] = books["goodreads_rating"] * books["goodreads_num_ratings"]

brush = alt.selection_interval(
    encodings=['x'] 
)

brushglobal = alt.selection_interval(
    resolve='global'
)

click = alt.selection_multi(fields = ['amazon_genre'], bind='legend')


genre_week = alt.Chart(books).mark_bar(
    opacity=0.6,
    binSpacing=0
).add_selection(
    brush,
    click
).encode(
    alt.X('nyt_weeks_on_list:Q',  bin=alt.BinParams(maxbins=30), axis=alt.Axis(title="Weeks on NYT")),
    alt.Y('count()', stack=None),
    alt.Color('amazon_genre'),
    opacity=alt.condition(click, alt.value(0.75), alt.value(0.05))
).properties(
    title = "Number of weeks fiction & non-fiction books on best-selling list",
    width=350
)

genre_pop = alt.Chart(books).mark_circle(size = 30 ,opacity = 0.3).add_selection(
    click,
    brushglobal
).encode(
    alt.X('total_star_goodreads:Q', axis=alt.Axis(title="Total stars Goodreads")),
    alt.Y('total_star_amazon:Q', axis=alt.Axis(title="Total stars Amazon")),
    alt.Color('amazon_genre', legend=alt.Legend(title="Amazon genre")),
    size = alt.condition(click, alt.value(30), alt.value(0)),
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.01))
).properties(
    title = "Popularity of fiction & non-fiction books on Amazon vs Goodreads"
).properties(
    width=350
)


mini_dashboard4 = alt.hconcat(genre_week.transform_filter(brushglobal), genre_pop)
mini_dashboard4

alt.HConcatChart(...)

## Task 5: How do the prices differ between Goodreads and Amazon platforms per book listed in Fiction/Non-fiction genre?

### Data wrangling for Task 5

In [12]:
column_subset = ['amazon_price', 'goodreads_price', 'amazon_genre', 'amazon_year']

# retain rows only if both amazon_price and goodreads_price are there, otherwise drop all NaN rows
books_prices = books.dropna(subset=column_subset, how='any')
# sort the books by title and amazon_prices
books_prices = books_prices.sort_values(by=["title", "amazon_price"], ascending=False)
# eliminate books that has 2 or more amazon_prices, retain only the highest price
books_prices = books_prices.drop_duplicates(subset=["title"])
# drops row where amazon_price or conlit price is 0
books_prices = books_prices[(books_prices["amazon_price"] > 0) & (books_prices["goodreads_price"] > 0)]
# compute the price differences in a column called price_diff
books_prices["price_diff"] = books_prices["amazon_price"] - books_prices["goodreads_price"]
books_prices["price_diff"] = books_prices["price_diff"].abs()
# amazon_year as a year
books_prices["amazon_year"] = books_prices["amazon_year"].dt.year
books_prices.head()

,title,amazon_author,amazon_rating,amazon_num_reviews,amazon_price,amazon_year,amazon_genre,conlit_genre,conlit_pubdate,conlit_author_gender,...,goodreads_likedPercent,goodreads_price,nyt_published_date,nyt_list_name_encoded,nyt_price,nyt_weeks_on_list,conlit_pubyear,total_star_amazon,total_star_goodreads,price_diff
113,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction,MIX,2013-01-01,F,...,89.0,4.88,2019-10-20,advice-how-to-and-miscellaneous,0.00,192.0,2013.0,67355.7,561437.20,3.12
365,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction,NaN,NaT,NaN,...,96.0,8.99,2019-12-29,series-books,0.00,561.0,NaN,46123.7,55424.42,0.99
107,Winter of the World: Book Two of the Century T...,Ken Follett,4.5,10760,15,2012,Fiction,BS,2012-01-01,M,...,98.0,5.80,2012-10-28,hardcover-fiction,36.00,4.0,2012.0,48420.0,549474.38,9.20
364,Wild: From Lost to Found on the Pacific Crest ...,Cheryl Strayed,4.4,17044,18,2012,Non Fiction,NaN,NaT,NaN,...,93.0,4.17,2012-09-16,hardcover-nonfiction,25.95,20.0,NaN,74993.6,2467878.31,13.83
363,Where the Wild Things Are,Maurice Sendak,4.8,9967,13,2009,Fiction,NaN,NaT,NaN,...,93.0,3.67,NaT,NaN,NaN,NaN,NaN,47841.6,3447461.48,9.33


In [13]:
# make the books_prices dataframe longer
books_prices_melt = pd.melt(books_prices, id_vars=['title', "price_diff", "amazon_genre", "amazon_year"], 
                            value_vars=['amazon_price', 'goodreads_price'])
# sort by price_diff + title, descending order
books_prices_melt = books_prices_melt.sort_values(by=["price_diff", "title"], ascending=False)
# remove the substring _price for readbility
books_prices_melt["variable"] = books_prices_melt["variable"].str.replace("_price", "")
books_prices_melt

,title,price_diff,amazon_genre,amazon_year,variable,value
139,Diagnostic and Statistical Manual of Mental Di...,98.84,Non Fiction,2013,amazon,105.00
308,Diagnostic and Statistical Manual of Mental Di...,98.84,Non Fiction,2013,goodreads,6.16
81,Percy Jackson and the Olympians Paperback Boxe...,84.87,Fiction,2010,amazon,2.00
250,Percy Jackson and the Olympians Paperback Boxe...,84.87,Fiction,2010,goodreads,86.87
61,The Book of Basketball: The NBA According to T...,47.42,Non Fiction,2009,amazon,53.00
...,...,...,...,...,...,...
221,The Five Dysfunctions of a Team: A Leadership ...,0.37,Non Fiction,2009,goodreads,6.37
165,"A Higher Loyalty: Truth, Lies, and Leadership",0.30,Non Fiction,2018,amazon,3.00
334,"A Higher Loyalty: Truth, Lies, and Leadership",0.30,Non Fiction,2018,goodreads,3.30
162,A Wrinkle in Time (Time Quintet),0.20,Fiction,2018,amazon,5.00


### Chart

In [14]:
sorted_title = books_prices_melt["title"].tolist()

genre_select = alt.selection_single(
    name="Select",
    fields=["amazon_genre", "amazon_year"],
    init={'amazon_genre': "Non Fiction", "amazon_year": 2017},
    bind={'amazon_genre': alt.binding_select(options=['Non Fiction', 'Fiction'], name = 'Amazon Publication Year: '),
          'amazon_year': alt.binding_select(
              options=books_prices["amazon_year"].sort_values(ascending=False).unique().tolist(), name = 'Amazon Genre: ')}
)

price_plot = alt.Chart(books_prices_melt).mark_point(
    filled=True, size=70
).add_selection(
    genre_select
).encode(
    alt.X("title", sort=sorted_title),
    alt.Y("value", scale=alt.Scale(zero=False)),
    alt.Shape("variable", scale=alt.Scale(range=["triangle", "circle"]), 
              legend=alt.Legend(title="Listing site")),
    alt.Color("variable", scale=alt.Scale(range=["#ff9900", "#75420e"])),
).transform_filter(
    genre_select
)

line_graph = alt.Chart(books_prices).mark_line().encode(
    alt.X("title", sort=sorted_title),
    alt.Y("amazon_price", title = 'Price Difference'),
    alt.Y2("goodreads_price"),
    tooltip=[alt.Tooltip("price_diff", title="Price Difference"),
            alt.Tooltip("amazon_price", title = "Price on Amazon"),
            alt.Tooltip("goodreads_price", title = "Price on Goodreads")]
).transform_filter(
    genre_select
)

alt.layer(price_plot, line_graph).properties(
    title="Price differences for books listed on Amazon vs Goodreads",
).properties(
    width=450
)

alt.LayerChart(...)

## Task 6

In [15]:

rating = ['amazon_rating', 'goodreads_rating']
bar = alt.Chart(books).mark_bar().encode(
    alt.Y('count()'),
    alt.X(alt.repeat("repeat"), type ='quantitative', bin=alt.BinParams(maxbins=30))
).properties(
    height=200,
    title ="Number of Books per Rating",
).repeat(
    repeat=rating,
    columns =1
)

brush = alt.selection_interval(
    resolve='global'
)


scatter = alt.Chart(books).mark_circle().add_selection(
    brush
).encode(
    alt.X('amazon_num_reviews:Q', axis=alt.Axis(title="Amazon Reviews")),
    alt.Y('goodreads_num_ratings:Q', axis=alt.Axis(title="Goodreads Reviews")),
    opacity=alt.condition(brush, alt.value(0.65), alt.value(0.03))
).properties(
    title ="Goodreads Reviews vs Amazon Reviews",
    height = 350,
    width = 350
)



mini_dashboard = alt.hconcat(scatter, bar.transform_filter(brush))
mini_dashboard

alt.HConcatChart(...)

## Task 7

In [16]:
books_awards = books.copy()
books_awards["goodreads_award"] = books_awards.goodreads_awards.apply(lambda x: ast.literal_eval(x))
books_awards['goodreads_num_awards'] = books_awards['goodreads_award'].str.len()
books_awards = books_awards.dropna(subset=["nyt_weeks_on_list"], how="any")

In [17]:
awards_bsl_boxplot = alt.Chart(books_awards).mark_boxplot(
    median={"color": "red"},
    extent=1.5,
    ticks= True,
).encode(
    alt.X("goodreads_num_awards", title = "Number of Goodreads Awards"),
    alt.Y("nyt_weeks_on_list", title = "Number of Weeks on New York Times Best Selling List")
).transform_filter(
    (datum.goodreads_num_awards < 9)
).properties(
    height = 350,
    width = 450,
    title = 'Number of Goodreads Awards vs Duration on NYT Best Selling List'
)

awards_bsl_boxplot

alt.Chart(...)